In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from sklearn import model_selection, linear_model
from sklearn.impute import KNNImputer, SimpleImputer

In [ ]:
# Path: data-processing.ipynb
# Read in the data
df = pd.read_csv('data/application_train.csv')
df.head(15)

In [ ]:
df2 = df.copy()
df2["TARGET"] = df2["TARGET"].replace([0, 1], ["Didn't Default", "Defaulted"])
fig = px.histogram(df2, x="TARGET", title="Number Defaulted vs Not Defaulted", histnorm="percent", labels={
    "TARGET" : "Defaulted vs Not Defaulted"
})
fig.show()

This chart shows that the dataset is heavily skewed in favor of people who didn't default on their loans, which is representative of the real world. That implies that high accuracy results aren't necessarily a sign of a working algorithm, and that we would need to use other metrics to make sure our models work properly.

In [ ]:
df2 = df.copy()
df2 = df2[np.abs(stats.zscore(df2["AMT_INCOME_TOTAL"]) < 5)]
fig = px.histogram(df2, x="AMT_INCOME_TOTAL", nbins=100, title='Income Distribution', labels={
    "AMT_INCOME_TOTAL" : "Income"
})
fig.show()
# Excluded incredibly large outliers

This chart just shows the income distribution of our dataset, without some heavy outliers with an absolute z-score of 5 or higher. The distribution matches up with a generic population distribution of incomes, which shows that our dataset is a good representation of larger populations.

In [ ]:
# Add cols to check continuous distributions in relation to those who did and didn't default
cols = ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "AMT_INCOME_TOTAL", "AMT_CREDIT", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_LAST_PHONE_CHANGE",
        "OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", "DEF_60_CNT_SOCIAL_CIRCLE"]
for col in cols:
    df2 = df.copy()
    df2 = df2[df2[col].notnull()]
    df2 = df2[np.abs(stats.zscore(df2[col]) < 5)]

    default = df2.copy()
    default = default[default["TARGET"] == 1]
    not_default = df2.copy()
    not_default = not_default[not_default["TARGET"] == 0]

    fig = go.Figure()
    #fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Histogram(x=not_default[col], name="Didn't Default", nbinsx=50, histnorm="percent"),
        #secondary_y=False
    )
    fig.add_trace(
        go.Histogram(x=default[col], name="Defaulted", nbinsx=50, histnorm="percent"),
        #secondary_y=False
    )

    fig.update_layout(
        title_text= ("Probability of " + col + " Given Default Status")
    )
    fig.update_yaxes(title_text="Probabilty Percentage [0, 100]")
    #fig.update_yaxes(title_text="Non-Default Count", secondary_y=True)
    fig.update_xaxes(title_text=col)
    fig.update_layout(barmode='overlay')    
    fig.update_traces(opacity=0.7)
    fig.show()

The following charts are overlayed histograms which divide the data by default status, with the orange being those who did default and the blue representing those who didn't default. All of the charts are normalized around the likelihood of any datapoint landing in a certain bin given a person's default status, as just measuring the total count isn't very useful since so many more people didn't default than those who did.

**EXT_SOURCE_1**
This chart shows that the EXT_SOURCE_1 metric predicts a higher value for those who are less likely to default, and does a fairly good job of splitting the data, although it's a very gradual split.

**EXT_SOURCE_2**
This chart shows that the EXT_SOURCE_2 metric predicts a higher value for those who are less likely to default, and can predict the likelihood of not defaulting pretty well, though it has a more uniform distribution for those who did end up defaulting.

**EXT_SOURCE_3**
EXT_SOURCE_3 also predicts a higher value for those who are less likely to default, and follows a similar distribution to EXT_SOURCE_1, with a higher emphasis on those who didn't default.

**AMT_INCOME_TOTAL**
From this chart, you can see that default status does not really affect the income of a person, and that could mean that income doesn't play as large of a role in this dataset.

**AMT_CREDIT**
The amount credited also doesn't seem to play that large of a role in dividing the data, but there is a small trend of people who get a higher credit loan default slightly less than those who get smaller loans.

**DAYS_BIRTH**
The amount of days since your birth seems to directly relate to a person's default status. The more days since you were born (the older you are), the less likely it seems you are to default.

**DAYS_EMPLOYED**
This distribution seems to have a some faulty data, because a lot of values are above 350,000, which implies that some people worked roughly 1000 years. Otherwise, the negative values seem to be correct, showing that people who have been unemployed for longer are more likely to default.

**DAYS_LAST_PHONE_CHANGE**
This chart shows that clients who defaulted on their loans were more likely to have changed their phone recently than those who didn't default.

**Social Circles**
All four of these metrics measure how risky a person's social circle is by gathering data on how many of their friends had defaulted or had payments past due. All four of these features seem to have little effect, with more significance coming from the ones that check if people in the client's social circle actually defaulted, rather than just having payments past due.

In [ ]:
# Add cols to check discrete distributions in relation to those who did and didn't default
cols = ["CODE_GENDER", "OCCUPATION_TYPE", "NAME_EDUCATION_TYPE", "NAME_HOUSING_TYPE", "ORGANIZATION_TYPE"]
for col in cols:
    df2 = df.copy()
    df2 = df2[df2[col].notnull()]

    default = df2.copy()
    default = default[default["TARGET"] == 1]
    not_default = df2.copy()
    not_default = not_default[not_default["TARGET"] == 0]

    fig = go.Figure()
    #fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Histogram(x=not_default[col], name="Didn't Default", nbinsx=50, histnorm="percent"),
        #secondary_y=False
    )
    fig.add_trace(
        go.Histogram(x=default[col], name="Defaulted", nbinsx=50, histnorm="percent"),
        #secondary_y=False
    )

    fig.update_layout(
        title_text= ("Probability of " + col + " Given Default Status")
    )
    fig.update_yaxes(title_text="Probability Percentage [0, 100]")
    #fig.update_yaxes(title_text="Non-Default Count", secondary_y=True)
    fig.update_xaxes(title_text=col, categoryorder="total descending")
    fig.show()

The following charts are similar to the overlaid histograms above, except while those displayed continuous data, these display categorical data and thus are separated into two bars per category, rather than overlaying 2 distributions on top of one another.

**CODE_GENDER**
The graph shows that there are more people who identified as female in the dataset, and overall, males were more likely to default on their loans.

**OCCUPATION_TYPE**
Overall, people with occupations which usually pay more seemed to default significantly less. Those occupations include: core staff, managers, high skill tech staff, accountants, medicine staff, and private service staff. The rest of the occupations were either about even or had higher proportions of people who defaulted.

**NAME_EDUCATION_TYPE**
Similar to occupation, people with a higher level of education ended up comprising a smaller portion of those who defaulted on their loans.

**NAME_HOUSING_TYPE**
Those who live in their own housing comprise a relatively smaller proportion of those who defaulted than those who didn't, while almost every other housing type did the opposite.

**ORGANIZATION_TYPE**
This dataset is harder to analyze as there are a lot of categories that divide the data, but it also displays the different organizations people are in and what percentage of those who defaulted and those who didn't came from each type of organization.